#  Le SVM ou l’hypothèse max-margin ☕️☕️

**<span style='color:blue'> Objectifs de la séquence</span>** 
* Comprendre&nbsp;:
    * la notion de marge (dans les données),
* Être sensibilisé&nbsp;:
    * à la notion de noyaux pour les problèmes non linéaires,
    * aux aspects de généralisation du choix *max-margin*.
    


 ----

## I. Introduction
L'hypothèse implicite derrière le *max-margin* est qu'entre deux frontières de même complexité, la plus robuste aux perturbations (dans le sens où si on perturbe un élément du jeu d'apprentissage, la probabilité qu'il change de classe est la plus faible) est la meilleure. L'idée fait sens car on peut supposer que les échantillons nouveaux peuvent être vus comme des perturbations des échantillons du jeu d'apprentissage. 

La frontière la plus robuste aux perturbations est celle qui maximise la distance entre le point le plus proche et elle-même. C'est l'hypothèse du *max-margin*.

La figure ci-dessous illustre cette idée. Nous avons deux frontières qui séparent sans faire d'erreur les deux classes. Cependant, l'une de ces frontières semble mauvaise alors que l'autre maximise la marge.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
np.random.seed(0)

dataset_size = 20

X = np.r_[np.random.randn(dataset_size, 2) - [2, 2], np.random.randn(dataset_size, 2) + [2, 2]]
Y = [0] * dataset_size + [1] * dataset_size

# figure number
fignum = 1

kernel = 'linear'
clf = svm.SVC(kernel=kernel)
clf.fit(X, Y)

# get the separating hyperplane
w = clf.coef_[0]
a = -w[0] / w[1]
xx = np.linspace(-5, 5)
yy = a * xx - (clf.intercept_[0]) / w[1]

# plot the parallels to the separating hyperplane that pass through the
# support vectors (margin away from hyperplane in direction
# perpendicular to hyperplane). This is sqrt(1+a^2) away vertically in
# 2-d.
margin = 1 / np.sqrt(np.sum(clf.coef_ ** 2))
yy_down = yy - np.sqrt(1 + a ** 2) * margin
yy_up = yy + np.sqrt(1 + a ** 2) * margin

# plot the line, the points, and the nearest vectors to the plane
plt.figure(figsize=(14, 8))
plt.clf()
plt.plot(xx, yy, 'k-', label='The decision boundary that maximizes the margin')
plt.plot(xx, yy_down, 'k--')
plt.plot(xx, yy_up, 'k--')

plt.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1], s=80,
            facecolors='none', zorder=10, edgecolors='k')

plt.scatter(X[:, 0], X[:, 1], c=Y, zorder=10, cmap=plt.cm.Paired,
            edgecolors='k')

plt.axis('tight')
x_min = -4.8
x_max = 4.2
y_min = -6
y_max = 6

XX, YY = np.mgrid[x_min:x_max:500j, y_min:y_max:500j]
Z = clf.predict(np.c_[XX.ravel(), YY.ravel()])

# Put the result into a color plot
Z = Z.reshape(XX.shape)
plt.pcolormesh(XX, YY, Z, cmap=plt.cm.Paired, shading='auto')

x = np.linspace(-5, 5, 100)
y = -(clf.coef_[0, 0]+30.5) * x / clf.coef_[0, 1]+13
plt.plot(x, y, label='A decision boundary with no error')

plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.legend()

plt.show()

## II. Un problème de classification linéaire

Le SVM est un classifieur linéaire. Soit $\mathcal{X}\subset\mathbb{R}^d$ nos variables d'entrées et $\mathcal{Y}=\{-1,+1\}$ l'espace de nos variables à prédire. Un classifieur linéaire sépare les éléments de notre jeu de données par un hyperplan. Comme vous avez pu le voir dans le TP précédent, un hyperplan décrit par le vecteur normal $w$ est défini par les solutions de l'équations suivantes :

$$\langle w, x\rangle = 0$$


Si le produit scalaire est positif, on dira que notre échantillon $x$ appartient à la classe positive et inversement.

Un classifieur linéaire peut donc être décrit de la manière suivante :


$$\begin{aligned}
h_w:\mathcal{X}&\mapsto\mathcal{Y}=\{-1,+1\}\\
x&\rightarrow \text{sign}(\langle w, x\rangle)
\end{aligned}$$


De la même manière que pour les TPs précédents, on peut introduire la notion de biais en rajoutant une dimension de $1$ aux vecteurs $x$.

**<span style='color:blue'> Petite question d'algèbre</span>** 
**Trouvez le projecteur orthogonal de $\mathcal{X}$ sur l'hyperplan décrit par le vecteur $w$, noté $\text{proj}_w(x)$. Démontrez que $\forall x\in\mathcal{X},\ \text{proj}_w(x)\in\{z:\langle w, z\rangle=0\}$ (autrement dit, démontrez que la projection de $x$ sur la frontière est bien sur la frontière).**



 ----

**<span style='color:blue'> Petite question d'algèbre 2</span>** 
**Montrer que $\text{proj}_w^2=\text{proj}_w$ (le carré est pris dans le sens de la composition). Cela vous semble-t-il logique ?**



 ----


### A. Le primal

Comme dit plus haut, on ne cherche pas n'importe quel hyperplan, mais bien celui qui rang la marge maximale. La marge est définie par la plus patite distance entre un point du jeu de données et la frontière de décision.

La distance d'un point à la frontière est donnée par $|\langle x_i, w\rangle|$ ($w$ unitaire). La quantité $y_i\langle x_i, w\rangle$ est positive et indique la distance à la frontière si le point est bien classé et donne la distance négative si le point est mal classé. Ainsi $\min_{i\leq m} y_i\langle x_i, w\rangle$ nous donne le point la plus petite distance (négative si mal classé).

On souhaite donc trouver $w$ tel que cette distance soit maximale (i.e. le point le plus proche est le plus loin possible de la frontière) :

$$\hat{w}=\text{argmax}_{w, \lVert w\rVert=1}\min_{i\leq m}y_i\langle x_i, w\rangle$$


Il est possible de montrer que le vecteur $w=w_0/\lVert w_0\rVert$ tel que :

$$w_0=\text{argmin}_{w}\lVert w\rVert^2_2,\ s.t. \forall i\leq m,\ y_i\langle x_i, w\rangle \geq 1$$


est solution de ce problème de minimisation.


**<span style='color:orange'> Preuve</span>** 
Soit 
$\boldsymbol{w}^\star$ une solution du premier problème et soit $\gamma^\star=\min_{i\leq m}y_i\langle\boldsymbol{x_i}, \boldsymbol{w^\star}\rangle$. $\gamma^\star$ est donc la distance du point le plus proche de l'hyperplan de vecteur normal $\boldsymbol{w^\star}$ à ce dernier. Nous avons donc

$$\forall i\leq m,\ y_i\langle \boldsymbol{x_i}, \boldsymbol{w^\star}\rangle \geq \gamma^\star,$$

et de manière totalement équivalente~:

$$\forall i\leq m,\ y_i\langle \boldsymbol{x_i}, \boldsymbol{w^\star}/{\gamma^\star}\rangle \geq 1$$

Notons ici que $\boldsymbol{w^\star}/{\gamma^\star}$ satisfait bien les contraintes du problèmes d'optimisation quadratique (le second problème). Ainsi, il suffit de montrer qu'il n'existe pas de meilleure solution au problème quadratique, pour que nos deux problèmes soient équivalents.

Soit $\boldsymbol{w_0}$ la solution du problème quadratique avant normalisation. On a ainsi $\lVert\boldsymbol{w_0}\rVert\leq \lVert\boldsymbol{w^\star/\gamma^\star}\rVert =1/\gamma^\star$. L'inégalité vient du fait que $\boldsymbol{w^\star/\gamma^\star}$ satisfait les contraintes mais n'est peut-être pas la meilleure solution. Soit $\hat{\boldsymbol{w}}=\boldsymbol{w_0}/\lVert\boldsymbol{w_0}\rVert$. Nous avons $\forall i\leq m$:


$$y_i\langle \boldsymbol{x_i}, \hat{\boldsymbol{w}}\rangle=\frac{1}{\lVert\boldsymbol{w_0}\rVert} y_i\langle \boldsymbol{x_i}, \boldsymbol{w_0}\rangle\geq \frac{1}{\lVert\boldsymbol{w_0}\rVert}\geq \gamma^\star.$$

La première inégalité vient de la contrainte du problème d'optimisation quadratique et la seconde que la solution du premier problème satisfait elle-même ces contraintes. Puisque $\lVert\hat{\boldsymbol{w}}\rVert=1$ et définit une marge au moins aussi grande que $\boldsymbol{w}^\star$ qui par définition définit la plus grande marge, les solutions sont équivalentes.



 ----

Remarquez que cela fait penser à la régularisation : parmi toutes les solutions possibles, on cherche celle de norme minimale.

### B. Le dual (optionnel)

Le problème d'optimisation si dessus est ce qu'on appelle un problème d'optimisation sous contrainte. Un tel problème est associé à ce qu'on appelle un Lagrangien :

$$\mathcal{L}(w, \alpha)=\frac{1}{2}\lVert w\rVert_2^2+\sum_{i=1}^m\alpha_i(1-y_i\langle x_i, w\rangle),\ \alpha_j\geq 0, j\leq m$$

Notons $g(w)=\max_{\alpha,\alpha\geq 0}\mathcal{L}(w,\alpha)$. On observe assez rapidement que $g(w)=\infty$ si une des contraintes n'est pas satisfaite et vaut $\lVert w\rVert^2_2/2$ sinon.

Ainsi, minimiser $g(w)$ revient à minimiser la norme du vecteur $w$ en respectant les contraintes. C'est ce qu'on appelle le *primal* qu'on note $p^\star$ :

$$p^\star=\min_w\max_{\alpha,\alpha\geq 0}\mathcal{L}(w,\alpha)$$

Le passage au dual permet d'inverser la minimisation et la maximisation. Il n'est pas évident de montrer que les deux problèmes sont équivalents. C'est ici le cas et on note $d^\star$ le dual (on parle donc de dualité forte) :

$$d^\star=\max_{\alpha,\alpha\geq 0}\min_w\mathcal{L}(w,\alpha).$$

**<span style='color:orange'> Détails</span>** 
où on peut vérifier qu'on a nécessairement $d^\star\leq p^\star$. Ainsi, dans le cadre du primal, le problème est une minimisation, mais devient une maximisation dans le dual. Le gap de dualité donne l'écart entre le primal et le dual. On parle de dualité faible si le gap est positif et de dualité forte s'il est nul. Il se trouve que si $\lVert\boldsymbol{w}\rVert^2$ et $\alpha_i(1 - y_i\langle \boldsymbol{x_i}, \boldsymbol{w}\rangle)$ sont convexes en $\boldsymbol{w}$ et $\alpha_i$ respectivement, et qu'il existe $\boldsymbol{w}$ tel que $1 - y_i\langle \boldsymbol{x_i}, \boldsymbol{w}\rangle<0$ (inégalité stricte impliquant que $\boldsymbol{w}$ est dans l'enveloppe affine de l'ensemble de faisabilité), alors nous avons une dualité stricte (conditions de Slater). On obtient donc~:

$$p^\star=\mathcal{L}(\boldsymbol{w^\star}, \boldsymbol{\alpha^\star})=d^\star.$$



 ----
Quelques éléments de calculs plus loin (le minimum est un point critique, on annule les dérivées partielles, etc.), on reformule le dual de la manière suivante :

$$\max_{\alpha,\alpha\geq 0}\sum_i\alpha_i-\frac{1}{2}\sum_i\sum_j\alpha_i\alpha_jy_iy_j\langle x_i, x_j\rangle$$

et

$$w=\frac{1}{2}\sum_i \alpha_iy_ix_i$$

**<span style='color:orange'> Détails</span>** 
Concentrons-nous sur l'expression duale. Pour un $\boldsymbol{\alpha}$ donné, $\min_{\boldsymbol{w}}\mathcal{L}(\boldsymbol{w}, \boldsymbol{\alpha})$ minimise $\mathcal{L}$ relativement à $\boldsymbol{w}$. Ainsi, nous avons:

$$0=\frac{\partial \mathcal{L}}{\partial\boldsymbol{w}}=\boldsymbol{w}-\sum_i\alpha_iy_i\boldsymbol{x_i}\Leftrightarrow\boldsymbol{w}=\sum_i\alpha_iy_i\boldsymbol{x_i}.$$


Autrement dit, nous pouvons reformuler le problème dual de la manière suivante:

$$\max_{\boldsymbol{\alpha}, \boldsymbol{\alpha}\geq \boldsymbol{0}}\sum_i\alpha_i-\frac{1}{2}\sum_i\sum_j\alpha_i\alpha_jy_iy_j\langle \boldsymbol{x_i}, \boldsymbol{x_j}\rangle.$$

Ce problème d'optimisation est équivalent au primal et consiste à maximiser une quantité qui ne dépend plus que du produit scalaire entre les points de notre jeu de données (à un signe près s'ils appartiennent à des classes opposées).



 ----
Ainsi, notre modèle prédictif prend la forme suivante :


$$\begin{aligned}
h:\mathcal{X}&\mapsto\mathcal{Y}\\
x&\rightarrow\text{sign}(\sum_i\alpha_iy_i\langle x_i, x\rangle)
\end{aligned}$$

### C. Via une *surrogate loss*

Nous avons vu que la minimisation empirique était difficile en générale. Pour cela, nous minimisions souvent une *surrogate loss*. Le SVM présenté à l'instant peut être reformulé au travers d'une *surrogate loss* appelée la *hinge loss*&nbsp;:

$$\ell^\text{hinge}(z)=\max(0;1-z).$$

Le problème du SVM devient alors&nbsp;:

$$\mathcal{L}(\omega)=\frac{1}{m}\sum_i \ell (y_i\langle \omega, x_i\rangle)+\lambda\lVert \omega\rVert_2$$

où nous avons rajouté une pénalité à la *hinge loss* afin de choisir le vecteur de paramètres de norme minimale.

## III. L'astuce du noyau (optionnel, suite du dual)

L'astuce du noyau découle de la formation duale et notamment du fait que celle-ci n'est liée aux données qu'au travers du produit $y_iy_i$ et du produit scalaire $\langle x_i, x_j\rangle$. Si le problème de classification est non linéaire, il est possible de passer par une transformation non linéaire $\phi:\mathcal{X}\mapsto\mathcal{F}$ de nos données d'entrées. Le problème devient donc :

$$\max_{\alpha,\alpha\geq 0}\sum_i\alpha_i-\frac{1}{2}\sum_i\sum_j\alpha_i\alpha_jy_iy_j\langle \phi(x_i), \phi(x_j)\rangle$$

Sans rentrer dans les détails, l'astuce du noyau vient de l'existence de fonctions :

$$\begin{aligned}
k:\mathcal{X}\times\mathcal{X}&\mapsto\mathbb{R}\\
x_i,x_j&\rightarrow k(x_i,x_j)=\langle\phi(x_i),\phi(x_j)\rangle.
\end{aligned}$$


Ces fonctions $k$ ne nécessitent pas de projeter les $x$ dans un espace de plus grande dimension et permettent d'obtenir le résultat du produit scalaire directement dans l'espace d'origine. Ainsi, on peut même calculer le produit scalaire dans des espaces de dimensions infinies.

Par exemple le noyau :

$$k(x_i, x_j)=(\langle x_i, x_j\rangle+c)^n,$$

nous permet de faire une transformations polynomiales de degré $n$ directement dans l'espace d'origine ; on remarque que la puissance $n$ est calculée sur le résultat du produit scalaire ($+c$) qui est donc un sclaire. Dans le cas où nos données seraient dans $\mathbb{R}^2$, prenons la fonction de transformation polynomiale suivante:

$$\phi(x)=[x_{1}^2,\sqrt{2}x_{1}x_{2}, x_2^2]^T.$$

Il s'agit du noyau avec $c=0$ et $n=2$.

Un autre noyau est le *noyau gaussien* définit comme:

$$k(x_i, x_j)=\text{exp}\Big(-\frac{\lVert x_i-x_j\rVert^2_2}{2}\Big)$$

où les prédictions dépendent des points dans un voisinage calculé par notre noyau gaussien.

Il se trouve que nous avons:



$$\text{exp}\Big(-\frac{\lVert x_i-x_j\rVert^2_2}{2}\Big)=C\sum_{l=0}^\infty \frac{\langle x_i, x_j\rangle^l}{l!}=C\sum_{l=0}^\infty \frac{k_{\text{poly}(l)}(x_i, x_j)}{l!},$$

où

$$C=\text{exp}\Big(-\frac{1}{2}\lVert x_i\rVert_2^2\Big)\text{exp}\Big(-\frac{1}{2}\lVert x_j\rVert_2^2\Big).$$

## IV. Visualisation de la frontière de décision

L'objectif de ce premier exercice est de visualiser la frontière de décision d'un SVM en jouant sur un exemple simple avec les noyaux offerts par la librairie $\texttt{scikit-learn}$.

La visualisation suivante permet d'observer la marge et notamment les vecteurs de supports.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
np.random.seed(0)

dataset_size = 20

X = np.r_[np.random.randn(dataset_size, 2) - [2, 2], np.random.randn(dataset_size, 2) + [2, 2]]
Y = [0] * dataset_size + [1] * dataset_size

# figure number
fignum = 1

kernel = 'linear'
clf = svm.SVC(kernel=kernel)
clf.fit(X, Y)

# get the separating hyperplane
w = clf.coef_[0]
a = -w[0] / w[1]
xx = np.linspace(-5, 5)
yy = a * xx - (clf.intercept_[0]) / w[1]

# plot the parallels to the separating hyperplane that pass through the
# support vectors (margin away from hyperplane in direction
# perpendicular to hyperplane). This is sqrt(1+a^2) away vertically in
# 2-d.
margin = 1 / np.sqrt(np.sum(clf.coef_ ** 2))
yy_down = yy - np.sqrt(1 + a ** 2) * margin
yy_up = yy + np.sqrt(1 + a ** 2) * margin

# plot the line, the points, and the nearest vectors to the plane
plt.figure(figsize=(14, 8))
plt.clf()
plt.plot(xx, yy, 'k-')
plt.plot(xx, yy_down, 'k--')
plt.plot(xx, yy_up, 'k--')

plt.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1], s=80,
            facecolors='none', zorder=10, edgecolors='k')

plt.scatter(X[:, 0], X[:, 1], c=Y, zorder=10, cmap=plt.cm.Paired,
            edgecolors='k')

plt.axis('tight')
x_min = -4.8
x_max = 4.2
y_min = -6
y_max = 6

XX, YY = np.mgrid[x_min:x_max:500j, y_min:y_max:500j]
Z = clf.predict(np.c_[XX.ravel(), YY.ravel()])

# Put the result into a color plot
Z = Z.reshape(XX.shape)
plt.pcolormesh(XX, YY, Z, cmap=plt.cm.Paired, shading='auto')

plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)

plt.xticks(())
plt.yticks(())
plt.show()


In [ ]:
def sample_data(size=200):
    X = np.random.uniform(-1, 1, size=(size, 2))
    y = X[:, 0]**3 < X[:, 1]
    return X, y
X, y = sample_data()

In [ ]:
def plot(X, y, clf=None):
    plt.figure(figsize=(14, 8))
    plt.xticks(())
    plt.yticks(())
    if clf is not None:

        XX, YY = np.mgrid[-1:1:500j, -1:1:500j]
        Z = clf.predict(np.c_[XX.ravel(), YY.ravel()])

        # Put the result into a color plot
        Z = Z.reshape(XX.shape)
        plt.pcolormesh(XX, YY, Z, cmap=plt.cm.Paired, shading='auto')

    plt.xlim(-1, 1)
    plt.ylim(-1, 1)

    plt.scatter(X[:, 0], X[:, 1], c=y)

    plt.show()

In [ ]:
plot(X, y)

Voici la visualisation d'un SVM avec un noyau linéaire (un produit scalaire $\langle\cdot,\cdot\rangle$).

In [ ]:
kernel = 'linear'
clf = svm.SVC(kernel=kernel)
clf.fit(X, y)

plot(X, y, clf)

Comme vous avez pu le voir, si vous avez lu la section concernant le dual, le SVM permet de remplacer les comparaisons linéaires (i.e. le produit scalaire), par des comparaisons non-linéaires (i.e. produit scalaire dans un espace où les données sont projetées non linéairement). La librairie $\texttt{scikir-learn}$ permet de jouer avec ce paramètre.

**<span style='color:blue'> Exercice</span>** **Jouez avec plusieurs noyaux et observez la forme de la frontière de décision.**


 ----

In [ ]:
####### Complete this part ######## or die ####################
...
...
...
...
###############################################################


**<span style='color:blue'> Question</span>** 
**Comparez la robustesse d'un SVM par rapport à un 1NN relativement à la dimension du problème. Le SVM est-il plus ou moins robuste que le 1NN ?**



 ----

In [ ]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

def sample_data(n, k=3, d=3, mu=1):
    y = np.random.randint(0, 2, size=(n, 1))
    
    X = np.random.normal(mu, 1, size=(n, k))
    X = y*X-(1-y)*X # positive have mean mu and negative, -mu
    noise = np.random.normal(0, 1, size=(n, d-k))
    X = np.concatenate([X, noise], axis=1)
    
    return X, y

scores_svm = []
scores = []
redo = 5
max_dim = 5000
first_dim = 10
steps = 100

for d in range(first_dim, max_dim, steps):
    s_svm = 0
    s = 0
    for _ in range(redo):
        X, y = sample_data(100, d=d)
        X_test, y_test = sample_data(200, d=d)
        
        model = SVC(kernel='linear', C=1.)
        model.fit(X, y.reshape((y.shape[0],)))
        s_svm += model.score(X_test, y_test.reshape((y_test.shape[0],)))/redo
        
        c = KNeighborsClassifier()
        c.fit(X, y.reshape((y.shape[0],)))
        s += c.score(X_test, y_test.reshape((y_test.shape[0],)))/redo
    scores.append(s)
    scores_svm.append(s_svm)
    
plt.plot(list(range(first_dim, max_dim, steps)), scores_svm, label='SVM')
plt.plot(list(range(first_dim, max_dim, steps)), scores, label='KNN')
plt.legend()
plt.show()

## V. Sur de vrais données

**<span style='color:blue'> Exercice</span>** 
**Utilisez le SVM, la régression logistique ou encore le KNN pour résoudre les problèmes ci-dessous.**



 ----

### Iris dataset

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()

X = iris['data']
y = iris['target']

In [ ]:
####### Complete this part ######## or die ####################
...
...
...
###############################################################


### Digits dataset

In [ ]:
from sklearn import datasets
digit = datasets.load_digits()

X = digit['data']
Y = digit['target']

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(14, 8))
fig.tight_layout()
for i in range(10):
    plt.subplot(2, 5, i+1)
    img = X[i].reshape((8, 8))
    plt.gca().set_axis_off()
    
    plt.imshow(img, cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title('Label: '+str(Y[i]))
plt.show()

In [ ]:
####### Complete this part ######## or die ####################
...
...
...
###############################################################


### Wine dataset

In [ ]:
from sklearn import datasets
wine = datasets.load_wine()

X = wine['data']
Y = wine['target']

In [ ]:
####### Complete this part ######## or die ####################
...
...
...
###############################################################


## VI. Majorant de généralisation pour le SVM

Nous avons vu dans la séquence traitant du modèle formel de l'apprentissage que nous pouvions majorer l'erreur attendue du minimiseur du risque empirique en espérance. Celle-ci dépendait de ce qu'on appelle la dimension VC. Pour un classifieur linéaire, la dimension VC est $d+1$ (i.e. le nombre de paramètres), où $d$ est la dimension de nos données.

Il se trouve que le minimiseur empirique est l'un des classifieurs linéaires de $\mathcal{H}$. Le SVM ne fait pas n'importe quel choix. La solution sera le classifieur linéaire qui maximise la marge. Cela nous permet d'obtenir le majorant suivant.

**<span style='color:blue'> Théorème (Majorant de généralisation du SVM)</span>** 
Soit $\mathbb{P}$ une distribution sur $\mathcal{X}\times\mathcal{Y}$ avec $\mathcal{Y}=\{-1,+1\}$ telle que $\exists w^\star$, $\lVert w^\star\rVert_2=1$, $y\langle w^\star, x\rangle+b\geq \gamma$ $\forall x, y$ et $\lVert x\rVert_2\leq \rho$ $\forall x$. Notons $h_\text{SVM}$ la solution du SVM. Soit $\delta>0$ Nous avons alors avec probabilité $1-\delta$&nbsp;:

$$L(h_\text{SVM})\leq \sqrt{\frac{4(\rho/\gamma)^2}{m}}+\sqrt{\frac{2\log(2/\delta)}{m}}$$



 ----
L'élément important de ce théorème est que la capacité de généralisation du SVM ne dépend plus de la dimension des données mais de la marge qui sépare nos données. La preuve de ce théorème est disponible dans&nbsp;:

*Shalev-Shwartz, Shai, et Shai Ben-David. Understanding Machine Learning: From Theory to Algorithms. Cambridge: Cambridge University Press, 2014.*
